In [71]:
import copy
import json
import math
import scipy
import statistics

In [2]:
ranked_data_path = f"bigresponse.json"
ranked_data_file = open(ranked_data_path,encoding="utf-8")
ranked_data = json.load(ranked_data_file)

In [211]:
players = ranked_data["players"]
maps = ranked_data["maps"]
scores = ranked_data["scores"]
filtered_scores = [score for score in scores if score["accuracy"] < 100]

In [212]:
players_by_id = {player["id"] : player for player in players}
maps_by_id = {bmap["id"] : bmap for bmap in maps}

In [213]:
scores_by_player_id = {}
scores_by_map_id = {}

def add_score(score):
    player_id = score["playerId"]
    map_id = score["leaderboardId"]
    
    player_map = scores_by_player_id.get(player_id,{})
    player_map[map_id] = score
    
    scores_by_player_id[player_id] = player_map
    
    map_map = scores_by_map_id.get(map_id,{})
    map_map[player_id] = score
    
    scores_by_map_id[map_id] = map_map
    
#for score in scores:
for score in filtered_scores:
    add_score(score)

In [200]:
modifiers = {"SF":1.2,
             "GN":1,
             "SA":1.05,
             "PM":1,
             "IF":1,
             "NO":0.5,
             "BE":1,
             "SS":0.75,
             "FS":1.05,
             "NB":0.5,
             "SC":1,
             "OD":1,
             "DA":1,
             "CS":1,
             "NA":0.5,
             "OP":0.5}
"""
"SF" - Super Fast,
"GN" - Ghost Notes,
"SA" - Strict Angles,
"PM" - Pro Mode,
"IF" - 1 life,
"NO" - No obstacles (no walls),
"BE" - Battery Energy,
"SS" - Slow song,
"FS" - Fast song,
"NB" - No bombs,
"SC" - Small Notes,
"OD" - Old Dots,
"DA" - Disappearing Arrows,
"CS" - ???,
"NA" - No Arrows -50%,
"OP" - Out of platform -50% 
"""

def modified_score(score,score_modifiers):
    final_score = score
    for modifier,multiplier in modifiers.items():
        if modifier in score_modifiers:
            if multiplier > 1:
                final_score = 1 - (1 - final_score)*(2 - multiplier)
            else:
                final_score = final_score * multiplier
            
    return final_score

In [7]:
"""
This is a mapping on the probability space rather than the population space.
That is, from (0,1) to (0,1). A probability distribution on the (0,1) interval.
We can do this with the Beta distribution, though other distributions might work.

I approximate a value for alpha and beta using this: https://homepage.divms.uiowa.edu/~mbognar/applets/beta.html
Trying to get the median around 0.9 and the probability close to 0 for 0.5, but a little bit higher for 0.65ish.
Beat Saber scores are typically at least 65%, on median probably around 0.9. They also must go down as the value
approaches 1, so beta must be greater than 1.

alpha = 10 and beta = 1.25 seemed to work quite well.
"""
beta_alpha = 10
beta_beta = 1.25

def beta_value(perc_value, beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.cdf(perc_value, beta_alpha, beta_beta)

def perc_beta_value(beta_value,beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.ppf(beta_value, beta_alpha, beta_beta)

"""
After renormalizing with the beta distribution, we apply the probit value, which assumes it's centred around 0.5.

We renormalize this to a value that is more typical of what we usually understand in Beat Saber.
For example, similar to star values. This is not really meant to be accurate, though.
"""

probit_mean = 7
probit_sd = 3

def probit_value(perc_value, probit_mean=probit_mean, probit_sd = probit_sd):
    return scipy.stats.norm.ppf(beta_value(perc_value),loc=probit_mean,scale=probit_sd)

def perc_probit_value(probit_value, probit_mean=probit_mean, probit_sd = probit_sd):
    return perc_beta_value(scipy.stats.norm.cdf(probit_value,loc=probit_mean,scale=probit_sd))

In [8]:
"""
We take the beta renormalization of probabilities above, and instead of using a probit function,
we map it to (0,inf) using a lognorm distribution.

There isn't a strong justification for this (other than the support), but there are some arguments.
What we want is to transform the probabilities into positive values that we can then multiply and divide in a way
that behaves well. Multiplying two values in the lognorm support produces a new lognorm value that is
the result of combining the other two in an independent way.
The shape of the lognorm also matches the way that we expect these values to look.
"""

# This parameterization is NOT a mistake. Read up on lognorm parameters and try to figure out why I did this.
# It isn't something terribly meaningful.
lognorm_mean = 7
lognorm_sd = 7
log_lognorm_sd = math.log(math.log(lognorm_sd))

def lognorm_value(perc_value, lognorm_mean=lognorm_mean, lognorm_sd=lognorm_sd, log_lognorm_sd=log_lognorm_sd):
    return scipy.stats.lognorm.ppf(beta_value(perc_value),s=log_lognorm_sd,loc=0,scale=lognorm_mean)

def perc_lognorm_value(lognorm_value, lognorm_mean=lognorm_mean, lognorm_sd=lognorm_sd, log_lognorm_sd=log_lognorm_sd):
    return perc_beta_value(scipy.stats.lognorm.cdf(lognorm_value,s=log_lognorm_sd,loc=0,scale=lognorm_mean))

In [9]:
"""
Very similar idea to the above, but with a maximum value.

There are some issues with this, like that a 100% score could lower a player's skill level
if it is on a very easy map. But this is very unlikely with the numbers, and 100% scores
are exceedingly rare anyway. There are also ways to deal with that later after estimating
difficulties.
"""

truncexp_max = 25
truncexp_base_mean = 7

def truncexp_value(perc_value,base_mean=truncexp_base_mean):
    return scipy.stats.truncexpon.ppf(beta_value(perc_value),b=truncexp_max,scale=base_mean)

def perc_truncexp_value(truncexp_value,base_mean=truncexp_base_mean):
    return perc_beta_value(scipy.stats.truncexpon.cdf(truncexp_value,b=truncexp_max,scale=base_mean))

In [18]:
"""
We normalize around a value of 7.
This means that a X star player on a 7 accability map achieves an X star score
and a 7 star player on an X accability map achieves an X star score.
"""

linear_mean = 7

def score_linear(skill,accability,linear_mean = linear_mean):
    return skill * accability / linear_mean

def accability_linear(skill,score,linear_mean = linear_mean):
    return score / skill * linear_mean

def skill_linear(accability,score,linear_mean = linear_mean):
    return score / accability * linear_mean

In [19]:
def abs_prop_error(value,evalue):        
    return abs((value-evalue)/evalue)

In [77]:
"""
Let's start with something simple: Just use the median instead of the average.
"""

def aggregation_median(scores,default_value=1):
    if scores == []:
        return default_value
    else:
        return statistics.median(scores)
    
def aggregation_median_f(default_value):
    def f(scores):
        return aggregation_median(scores,default_value)
    
    return f

In [98]:
"""
Do the average of the top percentage of scores.
"""
def aggregation_topscores(scores,perc=0.25,default_value=1):
    scores.sort()
    l = len(scores)
    n = math.floor(l*perc)
    
    if n == 0:
        return default_value
    else:
        return statistics.mean(scores[l-n:])
    
def aggregation_topscores_f(perc,default_value):
    def f(scores):
        return aggregation_topscores(scores,perc,default_value)
    
    return f

In [124]:
"""
Do the average of the bottom percentage of scores.
"""
def aggregation_bottomscores(scores,perc=0.25,default_value=1):
    scores.sort()
    l = len(scores)
    n = math.floor(l*perc)
    
    if n == 0:
        return default_value
    else:
        return statistics.mean(scores[0:n])
    
def aggregation_bottomscores_f(perc,default_value):
    def f(scores):
        return aggregation_bottomscores(scores,perc,default_value)
    
    return f

In [162]:
class BiPartiteStabilizer:
    """No description yet"""
    
    def __init__(self,anodes_ratings,bnodes_ratings,wdata,afun,bfun,wfun,aggregation_fun=aggregation_median,default_rating=1,max_iter=50,error_fun=abs_prop_error,error_aggregation_fun=aggregation_bottomscores):
        """
        anodes_ratings and bnodes_ratings must be dictionaries with the node identifiers as keys
        and initial ratings as values.
        
        wdata must be a doubly indexed dictionary with anode identifiers and bnode identifiers as respective indexes,
        respectively, and weights as values.
        
        afun and bfun must be functions taking a value of the other node type as first argument
        and a weight as the second argument, that returns the corresponding value for the node.
        
        Similarly, wfun must be a function that takes the value of an anode and bnode and returns the
        correct weight.
        
        These functions must be such that:
        - wfun(afun(b,w),b) = w
        - wfun(a,bfun(a,w)) = w
        - afun(bfun(a,w),w) = a
        - afun(b,wfun(a,b)) = a
        - bfun(afun(b,w),w) = b
        - bfun(a,wfun(a,b)) = b
        
        error_fun must take two parameters (actual value, expected value) and return a number indicating the error for that
        particular edge
        """
        self.anodes_ratings = anodes_ratings
        self.bnodes_ratings = bnodes_ratings
        
        self.adata = wdata
        self.bdata = self.process_bdata(wdata)
        
        self.afun = afun
        self.bfun = bfun
        self.wfun = wfun
        
        self.aggregation_fun = aggregation_fun
        
        self.error_fun = error_fun
        self.error_aggregation_fun = error_aggregation_fun
        
        self.default_rating = default_rating
        
        self.average_error = math.inf
        
        self.iter = 0
        self.max_iter = max_iter
        
        # Finish early when error increases
        self.last_average_error = math.inf
        self.last_anodes_ratings = anodes_ratings
        self.last_bnodes_ratings = bnodes_ratings        
    
    def process_bdata(self, wdata):
        bdata = {}
        
        for anode_id, anode_data in wdata.items():
            for bnode_id, w in anode_data.items():
                bnode_data = bdata.get(bnode_id,{})
                bnode_data[anode_id] = w
                bdata[bnode_id] = bnode_data
        
        return bdata       
        
    def acycle(self):
        calc_values = []
        for anode_id in self.anodes_ratings:
            calc_values.append(self.anode_process(anode_id))
            
        self.average_error = self.error_aggregation_fun(calc_values)
    
    def anode_process(self,anode_id):
        anode_data = self.adata[anode_id]
        
        calc_values = []
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            # asum += clamp(self.afun(bnode_value,w))
            # asum += self.afun(bnode_value,w)
            calc_values.append(self.afun(bnode_value,w))
        
        avg = self.aggregation_fun(calc_values)
        
        # self.anodes_ratings[anode_id] = clamp(avg)
        self.anodes_ratings[anode_id] = avg
                                              
        return self.anode_error(anode_id)
        
    def anode_error(self,anode_id):
        anode_data = self.adata[anode_id]
        anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
        
        calc_values = []
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            # calculated_w = clamp(self.wfun(anode_value,bnode_value))
            calculated_w = self.wfun(anode_value,bnode_value)
            # error = clamp(self.error_fun(calculated_w,w))
            error = self.error_fun(calculated_w,w)
            calc_values.append(error)
            
        return self.error_aggregation_fun(calc_values)
    
    def bcycle(self):
        for bnode_id in self.bnodes_ratings:
            self.bnode_process(bnode_id)
    
    def bnode_process(self,bnode_id):
        bnode_data = self.bdata[bnode_id]
        
        calc_values = []
        for anode_id, w in bnode_data.items():
            anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
            # bsum += clamp(self.bfun(anode_value,w))
            calc_values.append(self.bfun(anode_value,w))
        
        avg = self.aggregation_fun(calc_values)
        
        # self.bnodes_ratings[bnode_id] = clamp(avg)
        self.bnodes_ratings[bnode_id] = avg
    
    def save_last(self):
        self.last_anodes_ratings = self.anodes_ratings
        self.last_bnodes_ratings = self.bnodes_ratings
        self.last_average_error = self.average_error
        
    def restore_last(self):
        self.anodes_ratings = self.last_anodes_ratings
        self.bnodes_ratings = self.last_bnodes_ratings
        self.average_error = self.last_average_error
    
    def iterate(self):
        self.save_last()
        
        self.bcycle()
        self.acycle()
        
        self.iter += 1        
        print(f"Iteration {self.iter}, Average error: {self.average_error}")         
        
    def run(self):
        while (self.iter <= self.max_iter):            
            self.iterate()
            
            # Finish early with previous result if error went up
            if self.average_error > self.last_average_error:
                self.restore_last()
                print(f"Finishing early due to increased average error. Restoring previous values.")
                break
                
        return (self.anodes_ratings,self.bnodes_ratings)
    
    def test(self):
        return

In [214]:
#scores_doubly_indexed = {map_id : {player_id : clamp(lognorm_value(scores_by_map_id[map_id][player_id]["accuracy"])) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}
scores_doubly_indexed = {map_id : {player_id : truncexp_value(modified_score(scores_by_map_id[map_id][player_id]["accuracy"],scores_by_map_id[map_id][player_id]["modifiers"])) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}

In [215]:
default_rating = 7

map_ratings = {map_id : default_rating for map_id in scores_by_map_id}
player_ratings = {player_id: default_rating for player_id in scores_by_player_id}

# aggregation_fun = aggregation_median_f(default_rating)
aggregation_fun = aggregation_topscores_f(0.25,default_rating)
error_aggregation_fun=aggregation_bottomscores_f(0.5,0)

bps = BiPartiteStabilizer(map_ratings,player_ratings,scores_doubly_indexed,
                          accability_linear,skill_linear,score_linear,
                          aggregation_fun=aggregation_fun,
                          error_aggregation_fun=error_aggregation_fun,
                          max_iter=7)

bps.iterate()
#(maps_1,players_1) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))
#maps_1_errors = {key:bps.anode_error(key) for key in maps_1}
#bps.iterate()
#(maps_2,players_2) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))

(map_ratings,player_ratings) = bps.run()

Iteration 1, Average error: 0.17138069630977482
Iteration 2, Average error: 0.13296130231434303
Iteration 3, Average error: 0.1279573513541523
Iteration 4, Average error: 0.12905578777585325
Finishing early due to incrased average error. Restoring previous values.


In [115]:
maps_1_errors_outliers = {map_id:error for (map_id,error) in maps_1_errors.items() if error > 500}
print(maps_1_errors_outliers)

{'1442791': 726854.4327993888, '1375791': 1954.198324601896, '243e91': 13045.033069101148, '2c973xxxxx91': 6002.739983459292, '20fdbxxxxxxx91': 129008.97260961069, '23f7bxx11': 12825.025100115052, '1743411': 39399.519998481905, '16b071': 159799.76042189696, '13a071': 41954.65920044244, '102e231': 577252.3842709091, '102e211': 311739.9001775079, '101a031': 3159.0839786952447, '36bce91': 747.8643114786205, '1d05051': 135794.0385843516, '16abf11': 151027394456.47495, '35b17xxxx71': 1713.2255614606354, '36fb2xxxx71': 717.6964471752517, '349aexxxx11': 996.9658846817756, '242dc91': 2187.2734874165153, '33ce6xx11': 3046.0077066832887, '33aa2xxx91': 1408.9641899441783, '32331': 18110434413.11875, '31fbd91': 132630.6213413364, '1821811': 37911.181671025864, '30c0a91': 2615.5803492120103, '30c0a71': 242582.29475547257, '17a7051': 54876.17903080365, '242dc11': 808838.8439080734, '23f2e91': 88577.06480522401, '2698d11': 864.1416338763727, '2698d31': 804.7273412375035, '2f51': 685410.4742317508, '2

In [69]:
def errors_in_map(map_id,map_ratings,player_ratings):
    result = {}
    map_rating = map_ratings[map_id]
    for player_id,score in scores_doubly_indexed[map_id].items():
        player_rating = player_ratings[player_id]
        calculated_w = bps.wfun(map_rating,player_rating)
        error = bps.error_fun(calculated_w,score)
        result[player_id] = {"calculated_w":calculated_w,"actual_score":score,"error":error}
        
    return result

In [120]:
errors_this_map = errors_in_map("1442791",maps_1,players_1)
error_outliers = {player_id:error for (player_id,error) in errors_this_map.items() if error["error"] < 1}
print(error_outliers)

{'69': {'calculated_w': 11.061990892657283, 'actual_score': 6.397005318432781, 'error': 0.7292452236646553}, '1192': {'calculated_w': 4.140064631395779, 'actual_score': 3.576757724186184, 'error': 0.15749093191313748}, '1710': {'calculated_w': 1.815531364648208, 'actual_score': 1.595199242380236, 'error': 0.13812200784348994}, '2042': {'calculated_w': 6.62477031495925, 'actual_score': 3.8430901864780918, 'error': 0.7238133880564386}, '433': {'calculated_w': 2.6466629581969707, 'actual_score': 2.830051163924051, 'error': 0.06480031458964877}, '873': {'calculated_w': 4.313009103676543, 'actual_score': 4.175028686830819, 'error': 0.033048974556977743}, '286': {'calculated_w': 9.38371660883764, 'actual_score': 10.157720685551382, 'error': 0.07619859815742985}, '2711': {'calculated_w': 5.84567324163301, 'actual_score': 6.526356449347979, 'error': 0.10429758365143742}, '76561199013102210': {'calculated_w': 3.1250660519863844, 'actual_score': 1.9815975021255445, 'error': 0.5770437985687343}, 

In [66]:
scores_by_map_id["2c803x91"]["76561198279631500"]

{'id': 9961190,
 'leaderboardId': '2c803x91',
 'accuracy': 0.25158915,
 'modifiers': '',
 'playerId': '76561198279631500'}

In [88]:
hard_maps = {map_id : rating for (map_id,rating) in maps_1.items() if rating < 2}
print(hard_maps)

{'2dd6cxx92': 1.4587110411680493, '3697axxxx91': 1.86881632215721, '3aa79xxxxxxxxx91': 1.8343712645497328, '2c803x91': 1.9003041230627646, '194291': 1.7897381555593377, '41f391': 1.8307314599511388, '6b5f71': 1.7880064936416402}


1.8374421209050689

In [22]:
map_outliers = {map_id : rating for (map_id,rating) in map_ratings.items() if rating > 20}
player_outliers = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 20}

In [23]:
print(map_outliers)
print(player_outliers)

{'38670xxxx51': 20.499486537656807, '35d5bx51': 21.87754497289034, '37090x11': 20.68220115301559, '39ba2xxx51': 21.037262740611773, '2a57111': 23.372915897156656, '3b37bx51': 20.874448114112464, '3bb99x11': 21.33343685300185, '7e8f11': 20.538940694249117, '17dc431': 20.05791244383166}
{'76561198180044686': 20.185025990538136, '76561198306107330': 21.148122599348934, '76561198145281261': 20.32951576580459, '76561198988695829': 22.854734540730043, '76561199104169308': 20.913106078017822, '76561199031414897': 20.215610845092343, '76561199085118735': 22.60310784908709, '76561198166061709': 23.143759146437937, '2769016623220259': 21.826717193025022, '76561199081029968': 20.41228166231709, '76561198404774259': 21.540994640784945, '76561198333869741': 22.001659235037142, '76561199465530115': 22.030494662682248, '76561198960449289': 20.687624752849846, '76561197994319066': 21.135565793173377, '76561199266548375': 22.01953676701762}


In [221]:
hard_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 2.4}
print(hard_maps)

{'3697axxxx91': 2.30917332146772, '3aa79xxxxxxxxx91': 2.190340909321467, '2c803x91': 2.3782036745464117, '3b2bcxxxx91': 2.2950162273534525, '387a0xxxx91': 2.2860896474204195}


In [141]:
easy_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating > 18}
print(easy_maps)

{'2a57111': 21.005482139149965, '3bb99x11': 18.69288820952263, '7e8f11': 19.320317739667743}


In [173]:
good_players = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 22}
print(good_players)

{'3225556157461414': 22.16999368045714, '76561198988695829': 22.339952628037413}


In [225]:
#print(scores_doubly_indexed["7e8f11"])
#print(maps_by_id["2c803x91"])
print(players_by_id["76561199108348236"])

{'name': 'UglyApe', 'country': 'GB', 'id': '76561199108348236', 'avatar': 'https://cdn.assets.beatleader.xyz/76561199108348236R27.png'}


In [224]:
print(player_ratings["76561199108348236"])

9.342766569321332


In [226]:
print(scores_by_player_id["76561199108348236"])

{'1232e91': {'id': 3524623, 'leaderboardId': '1232e91', 'accuracy': 0.71168214, 'modifiers': '', 'playerId': '76561199108348236'}, '1d0ca91': {'id': 5705991, 'leaderboardId': '1d0ca91', 'accuracy': 0.8916043, 'modifiers': 'FS', 'playerId': '76561199108348236'}, '2538591': {'id': 11596786, 'leaderboardId': '2538591', 'accuracy': 0.8693901, 'modifiers': '', 'playerId': '76561199108348236'}, '35340xxx91': {'id': 12536001, 'leaderboardId': '35340xxx91', 'accuracy': 0.93467724, 'modifiers': '', 'playerId': '76561199108348236'}, '243e5xx91': {'id': 11598603, 'leaderboardId': '243e5xx91', 'accuracy': 0.91663027, 'modifiers': '', 'playerId': '76561199108348236'}, '14af891': {'id': 6159324, 'leaderboardId': '14af891', 'accuracy': 0.9177346, 'modifiers': '', 'playerId': '76561199108348236'}, '343a2xxx91': {'id': 15367652, 'leaderboardId': '343a2xxx91', 'accuracy': 0.93719846, 'modifiers': 'SS', 'playerId': '76561199108348236'}, '25b7d91': {'id': 8295466, 'leaderboardId': '25b7d91', 'accuracy': 0

In [177]:
print(map_ratings["2ee3bxxxxxx91"])

2.989550902060393


In [ ]:
filtered_map_ratings = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 3.69 and rating > 3.68}
print(filtered_map_ratings)

In [ ]:
print(len(scores))
print(len(filtered_scores))

In [ ]:
print(scores_by_player_id["76561198167588802"])

In [ ]:
print(truncexp_value(0.94))
print(perc_truncexp_value(12))

In [ ]:
print(skill_probspace_truncexp(0,25))

In [ ]:
print(score_probspace_truncexp(68,68))

In [ ]:
print(skill_linear(1,25))

In [105]:
print(aggregation_topscores([1,2,3,4,5,6,7,8,9,10,11,12,13,14,100]))

42.333333333333336


In [106]:
[1,2,3,4][0:2]

[1, 2]

In [117]:
a = [3,1,2,4]
a.sort()
print(a)

[1, 2, 3, 4]


In [123]:
error_aggregation_fun([0.1,0.2,0.3,0.4,100])

100

In [178]:
modified_scores = [score for score in scores if score["modifiers"] != "" and score["accuracy"] < 100]

In [180]:
modifier_combos = set([score["modifiers"] for score in modified_scores])
print(modifier_combos)

{'SF,GN,SA,PM,IF', 'GN,SA,PM', 'SS,PM,IF', 'SF,SC', 'DA,SA,PM', 'NB,NO,PM', 'NB,NO,IF', 'SF,GN,SA,SC,PM,BE', 'SS,NB,NO,SA,PM', 'GN,IF', 'SF,GN,SC,IF', 'DA,SF,PM,IF,OP', 'SA,PM,OP', 'DA,FS,PM', 'DA,SF,SC,PM,IF', 'SC,IF', 'SF,PM,IF', 'FS,SA,PM', 'FS,BE', 'FS,IF', 'SF,SA,SC,PM', 'SS,NA', 'NA,OP', 'DA,SS,OD', 'SC,NA', 'FS,GN,NA', 'SF,GN,SA,SC,PM', 'SS,OD', 'SF,SC,GN', 'DA,OD', 'SS,PM', 'SF,SC,BE', 'DA,SC,BE', 'SF,NA,NB,NO', 'NO,BE', 'DA,FS,BE,OP', 'DA,SA,SC,PM,IF', 'SS,NA,PM', 'FS,GN,BE', 'SF,NB,NO', 'SS,NA,OD', 'DA,FS,SA,SC,PM', 'GN', 'SF,PM,OP', 'DA,NB', 'SS,NA,NB,NO,IF', 'SS,SA,PM', 'DA,SF,OD', 'FS,PM,BE', 'DA,SS', 'BE,OP', 'SF,SA,SC,PM,BE', 'SF,GN,SC', 'FS,BE,OD', 'DA,PM', 'SF,GN', 'DA,SC,OP', 'SF,SA,PM,IF', 'SF,GN,PM,IF', 'DA,SF,SC,PM,BE', 'FS,NA,NO', 'DA,FS,SC,PM,BE', 'DA,PM,IF', 'SF,SC,PM', 'DA,FS,OD', 'SA,SC,PM', 'SF,SA,SC,PM,IF', 'SS,SA,SC,PM', 'SF,GN,PM,OP', 'SF,GN,OP', 'SS,NA,NB,NO,OD', 'BE,OD', 'SF,NB', 'SF,NO', 'GN,NB,NO,SA,PM', 'FS,NA', 'NA,NB,PM', 'DA,FS,BE', 'DA,SF,PM,IF', 

In [207]:
print(modified_score(0.95,"DA,SF,NO"))

0.48


In [192]:
modifiers = ["SF","GN","SA","PM","IF","NO","BE","SS","FS","NB","SC","OD","DA","CS","NA","OP"]
remaining_modifier_combos = modifier_combos
for modifier in modifiers:
    remaining_modifier_combos = [modifier_combo.replace(modifier+",","") for modifier_combo in remaining_modifier_combos]
    remaining_modifier_combos = [modifier_combo.replace(modifier,"") for modifier_combo in remaining_modifier_combos]
    remaining_modifier_combos = set(remaining_modifier_combos)    
    if "" in remaining_modifier_combos:
        remaining_modifier_combos.remove("")    
    
print(remaining_modifier_combos)

set()


TypeError: unsupported operand type(s) for -: 'str' and 'str'